In [1]:
# module imports

from configparser import ConfigParser
from mysql.connector import MySQLConnection,Error

import gzip
import json

In [2]:
# reading database credentials from config.ini file

def read(filename='config.ini',section='mysql'):
    parser=ConfigParser()
    parser.read(filename)
    
    db={}
    
    if parser.has_section(section):
        items=parser.items(section)
        for item in items:
            db[item[0]]=item[1]
    else:
        raise Exception(f'{section} not found in file {filename}')
    return db 


print(read(filename="config.ini",section="mysql"))

{'host': '127.0.0.1', 'port': '99966', 'database': 'goodreads', 'user': 'root', 'password': 'Milla123!'}


In [3]:
# connecting with MySQL/MariaDB database server and getting the connection and cursor object

def connect(creds):
    con=None
    try:
        print('Connecting to MySQL database...')
        con=MySQLConnection(**creds)
        
        
        if con.is_connected():
            print('Connection established')
            cus = con.cursor(buffered=True)
        else:
            print('Connection failed')
            
    except Error as e:
        print(e)
    finally:
        return con,cus
    
cn,cs=connect(creds=read(filename="config.ini",section="mysql"))

Connecting to MySQL database...
Connection established


In [4]:
# required columns

required_columns = [
"item_id",
"book_id",
"title",
"mod_title",
"ratings_count",
"average_rating",
"link",
"url",
"image_url",
"publication_day",
"publication_month",
"publication_year",
"num_pages",
"isbn",
"isbn13",
"description",
"publisher"]

# columns that need to be transformed into int

trans_to_int = [
    "book_id",
    "publication_day",
    "publication_month",
    "publication_year",
    "num_pages",
]

In [5]:
# json parser function into python dictionary

def parse_fields(line):
    data = json.loads(line)
    data_dict = {required_columns[0] : None}

    for col in required_columns[1:]:
        data_dict[col] = data[col]
    
    return data_dict

In [6]:
# reading the first line/record from the compressed json file in a streaming fashion

with gzip.open("../Datasets/Processed/itemDetails_RatingCount_gt15_p2.json.gz") as f:
    line = f.readline()

line

b'{"isbn":"","average_rating":4.03,"similar_books":["19997","828466","1569323","425389","1176674","262740","3743837","880461","2292726","1883810","1808197","625150","1988046","390170","2620131","383106","1597281"],"description":"Omnibus book club edition containing the Ladies of Madrigyn and the Witches of Wenshar.","link":"https:\\/\\/www.goodreads.com\\/book\\/show\\/7327624-the-unschooled-wizard","authors":[{"author_id":"10333","role":""}],"publisher":"Nelson Doubleday, Inc.","num_pages":"600","publication_day":"","publication_month":"","publication_year":"1987","isbn13":"","url":"https:\\/\\/www.goodreads.com\\/book\\/show\\/7327624-the-unschooled-wizard","image_url":"https:\\/\\/images.gr-assets.com\\/books\\/1304100136m\\/7327624.jpg","book_id":"7327624","ratings_count":140,"title":"The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)","mod_title":"the unschooled wizard sun wolf and starhawk 12"}\r\n'

In [7]:
# parsing json record into dictionary and storing that

test_record = parse_fields(line)
test_record

{'item_id': None,
 'book_id': '7327624',
 'title': 'The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)',
 'mod_title': 'the unschooled wizard sun wolf and starhawk 12',
 'ratings_count': 140,
 'average_rating': 4.03,
 'link': 'https://www.goodreads.com/book/show/7327624-the-unschooled-wizard',
 'url': 'https://www.goodreads.com/book/show/7327624-the-unschooled-wizard',
 'image_url': 'https://images.gr-assets.com/books/1304100136m/7327624.jpg',
 'publication_day': '',
 'publication_month': '',
 'publication_year': '1987',
 'num_pages': '600',
 'isbn': '',
 'isbn13': '',
 'description': 'Omnibus book club edition containing the Ladies of Madrigyn and the Witches of Wenshar.',
 'publisher': 'Nelson Doubleday, Inc.'}

In [8]:
# check the datatype of each field and the data as well

for col in required_columns:
     print(f"{col} - ",type(test_record[col])," - ", test_record[col])

item_id -  <class 'NoneType'>  -  None
book_id -  <class 'str'>  -  7327624
title -  <class 'str'>  -  The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)
mod_title -  <class 'str'>  -  the unschooled wizard sun wolf and starhawk 12
ratings_count -  <class 'int'>  -  140
average_rating -  <class 'float'>  -  4.03
link -  <class 'str'>  -  https://www.goodreads.com/book/show/7327624-the-unschooled-wizard
url -  <class 'str'>  -  https://www.goodreads.com/book/show/7327624-the-unschooled-wizard
image_url -  <class 'str'>  -  https://images.gr-assets.com/books/1304100136m/7327624.jpg
publication_day -  <class 'str'>  -  
publication_month -  <class 'str'>  -  
publication_year -  <class 'str'>  -  1987
num_pages -  <class 'str'>  -  600
isbn -  <class 'str'>  -  
isbn13 -  <class 'str'>  -  
description -  <class 'str'>  -  Omnibus book club edition containing the Ladies of Madrigyn and the Witches of Wenshar.
publisher -  <class 'str'>  -  Nelson Doubleday, Inc.


In [9]:
# accessing the fields dynamically and checking the length of each data so that we can understand which field's data is missing

for col in required_columns:
    if (len(str(test_record[col])) < 1):
        print(f"{col} = ", test_record[col])
    else:
        print(f"{col} = ",test_record[col])

item_id =  None
book_id =  7327624
title =  The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)
mod_title =  the unschooled wizard sun wolf and starhawk 12
ratings_count =  140
average_rating =  4.03
link =  https://www.goodreads.com/book/show/7327624-the-unschooled-wizard
url =  https://www.goodreads.com/book/show/7327624-the-unschooled-wizard
image_url =  https://images.gr-assets.com/books/1304100136m/7327624.jpg
publication_day =  
publication_month =  
publication_year =  1987
num_pages =  600
isbn =  
isbn13 =  
description =  Omnibus book club edition containing the Ladies of Madrigyn and the Witches of Wenshar.
publisher =  Nelson Doubleday, Inc.


In [10]:
# constructing the data transformation function for the fields with missing data

# taking 3 parameters
# all the column name list, colums that need casting to int and a single record/data that needs transformation
def transform_fields(column_names,trans_to_int,record):
    # looping through all the columns
    for col in column_names:
        # checking if the length of the data is less than 1 => means data is missing
        if (len(str(record[col])) < 1):
        # based on column, we will fill the place with appropriate place holder 
            if (col == "ratings_count"):
                record[col] = -1
            elif (col == "average_ratings"):
                record[col] = -1.0
            else:
                record[col] = "-1"
        # if data is not missing then converting the data into int if the selected column belongs to the column list trans_to_int 
        else:
            if col in trans_to_int:
                record[col] = int(record[col])
    return record

In [11]:
# testing transformation function

transform_fields(required_columns, trans_to_int, test_record)

{'item_id': None,
 'book_id': 7327624,
 'title': 'The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)',
 'mod_title': 'the unschooled wizard sun wolf and starhawk 12',
 'ratings_count': 140,
 'average_rating': 4.03,
 'link': 'https://www.goodreads.com/book/show/7327624-the-unschooled-wizard',
 'url': 'https://www.goodreads.com/book/show/7327624-the-unschooled-wizard',
 'image_url': 'https://images.gr-assets.com/books/1304100136m/7327624.jpg',
 'publication_day': '-1',
 'publication_month': '-1',
 'publication_year': 1987,
 'num_pages': 600,
 'isbn': '-1',
 'isbn13': '-1',
 'description': 'Omnibus book club edition containing the Ladies of Madrigyn and the Witches of Wenshar.',
 'publisher': 'Nelson Doubleday, Inc.'}

In [12]:
# SQL insert query => insert one record at a time

# insert function takes following parameters
# the non-transformed raw record as dictionary
# connection and cursor object 

# name of the table
table = "item"

def insert_one(record, req_columns, table=table, cn=cn, cs=cs):
    columns_string = """
    item_id,
    book_id,
    title,
    mod_title,
    ratings_count,
    average_rating,
    link,
    url,
    image_url,
    publication_day,
    publication_month,
    publication_year,
    num_pages,
    isbn,
    isbn13,
    description,
    publisher"""

    # placeholders as string
    place_holders = """
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s"""

    # transforming the raw dictionary record to make it suitable for the database upload
    record_dict = transform_fields(req_columns, trans_to_int, record)

    # accessing the dict data to prepare the data as tuple
    data_tuple = tuple(record_dict.values())

    # SQL query to insert the data
    sql_query = f"""
    INSERT INTO {table} ({columns_string})
    VALUES ({place_holders});
    """

    # trying to execute the query
    try:
        cs.execute(sql_query, data_tuple)
        cn.commit()
    # throwing error in case of unsuccessful attempt
    except Error as e:
        raise Exception(f"{e}")

In [13]:
# testing the data insert function

# insert_one(test_record,required_columns)

- the following code took 37m to execute

In [15]:
# we will go through all the books from the dataset and insert them one after the other

books = []

with gzip.open("../Datasets/Processed/itemDetails_RatingCount_gt15_p2.json.gz") as f:
    while True:
        # reading the line as json
        line = f.readline()

        # we will break the infinite loop when we reach the end of the dataset file
        if not line:
            break
        
        # parsing the json line into dictionary
        record_dict = parse_fields(line)

        # insert function to upload the data into database
        insert_one(record_dict,required_columns)